In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
file = 'RELATHE.csv'
df_main = pd.read_csv(file)
df_main = df_main.sample(frac = 1)
df_main.shape

(1427, 4323)

In [3]:
df_main.head

<bound method NDFrame.head of       V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V4314  V4315  V4316  \
227    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
1212   0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
835    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
1132   0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
257    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...    ...    ...    ...   
1086   0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
559    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
823    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
344    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
107    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   

      V4317  V4318  V4319  V4320  V4321  V4322  TARGET  
227       0 

In [4]:
#DATA DIVISION

df_1 = df_main.iloc[:300, :]
x_1 = df_1.drop('TARGET', axis = 1)
y_1 = df_1['TARGET']
df_2 = df_main.iloc[300:600, :]
x_2 = df_2.drop('TARGET', axis = 1)
y_2 = df_2['TARGET']
df_3 = df_main.iloc[600:900, :]
x_3 = df_3.drop('TARGET', axis = 1)
y_3 = df_3['TARGET']
df_4 = df_main.iloc[900:1200, :]
x_4 = df_4.drop('TARGET', axis = 1)
y_4 = df_4['TARGET']
df_5 = df_main.iloc[1200:, :]
x_5 = df_5.drop('TARGET', axis = 1)
y_5 = df_5['TARGET']
df_list = [df_1, df_2, df_3, df_4, df_5]
x_list = [x_1, x_2, x_3, x_4, x_5]

In [5]:
global_model = pickle.load(open("xgb_clf.dat", "rb"))

In [7]:
class Server():
    def __init__(self, df, model):
        self.df = df
        self.model = model
    def update_weights():
        return

server = Server(df_main, global_model)

In [16]:
class Client():
    def __init__(self, cli, df, model):
        self.cli = cli
        self.df = df
        self.model = model
    def generate_weights(self, model):
        df = self.df
        X = df.drop('TARGET', axis = 1)
        y = df['TARGET']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        eval_set = [(X_test, y_test)]
        model.fit(X_train, 
                y_train, 
                early_stopping_rounds=15,                  
                eval_set=eval_set,
                verbose = False)
        results = model.evals_result()
        return model, sum(results['validation_0']['logloss'])/len(results['validation_0']['logloss'])
    def inference(self, model):    #return the acc, loss of given model on local dataset
        df = self.df
        X = df.drop('TARGET', axis = 1)
        y = df['TARGET']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        y_pred = model.predict(X_test)
        return accuracy_score(y_test, y_pred), 

In [17]:
def update_model(global_model, local_weights):    #change the weights in the model itself
    # local weights is also a model
    global_new_model = global_model
    return global_new_model

In [18]:
cl1 = Client(1, df_1, global_model)
cl2 = Client(2, df_2, global_model)
cl3 = Client(3, df_3, global_model)
cl4 = Client(4, df_4, global_model)
cl5 = Client(5, df_5, global_model)

cli_list = [cl1, cl2, cl3, cl4, cl5]

In [20]:
c = 1    #Fraction of Clients to be used
global_training_rounds = 4
train_loss = []
train_accuracy = []
for i in range(global_training_rounds):
    m = max(c*5, 1)
    st = np.random.choice(5, m, replace = False)    #random set of m clients
    local_w = []
    local_loss = []
    print('Global Training Round: ', end = "")
    print(i+1)
    for k in st:
        print('Training | Client index: ', end = "")
        print(k+1)
        w, loss = cli_list[k].generate_weights(global_model)
        local_w.append(w)
        local_loss.append(loss)
        #w(t+1)k = clientupdate(k, wt)
        #w(t+1) = sum((nk/n) * w(t+1)k)
    global_model = update_model(global_model, local_w)
    train_loss.append(sum(local_loss)/len(local_loss))
    # Calculate avg training accuracy over all users at every epoch
    list_acc = []
    for j in range(5):
        acc = cli_list[j].inference(global_model)
        list_acc.append(sum(acc))
    train_accuracy.append(sum(list_acc)/len(list_acc))
    print("Training loss for Global Training Round: %2d is : %2.4f" % (i+1, train_loss[i]))
    print("Training accuracy for Global Training Round: %2d is : %2.4f" % (i+1, train_accuracy[i]))

Global Training Round: 1
Training | Client index: 5


C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Training | Client index: 4


C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Training | Client index: 1


...

Training | Client index: 4


C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Training | Client index: 2


C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Accuracy | Client index: 1
Accuracy | Client index: 2
Accuracy | Client index: 3
Accuracy | Client index: 4
Accuracy | Client index: 5
Training loss for Global Training Round:  4 is : 0.4883
Training accuracy for Global Training Round:  4 is : 0.7722


In [ ]:
#Find test accuracy and loss

In [ ]:
list_acc = []
acc = cli_list[2].inference(global_model)
list_acc.append(acc)

In [ ]:
list_acc

In [ ]:
trees = global_model.get_booster().trees_to_dataframe()

In [ ]:
trees

In [ ]:
trees.loc[trees['Feature'] == 'Leaf']["Gain"]

In [ ]:
#Possible Loophole: Using full batch instead of splitting in client_update [fedavg, mcmahan]

In [ ]:
dump_list = global_model.get_booster().get_dump()
len(dump_list)